In [19]:
import pandas as pd
import numpy as np

df = pd.read_excel("pbmc_sample_list_raw.xlsx")

In [20]:
df

,Enrollment Number,Gender,Ethnicity,Healthy Control or HS?
0,HS-042,F,Caucasian,HS
1,HS-110,F,Caucasian,HS
2,HS-129,F,African American,HS
3,HS-132,F,Asian American,HS
4,HS-157,F,Caucasian,HS
...,...,...,...,...
137,WSU-119,M,African American,HS
138,WSU-122,F,African American,HS
139,WSU-124,F,African American,HS
140,WSU-126,F,African American,HS


In [21]:
# change name of first enrollment number to match up with rocks HS-042 -> HS-42
df.at[0, "Enrollment Number"] = "HS-42"
df["SampleID"] = df["Enrollment Number"].apply(lambda x: x.replace('-', '_'))

In [22]:
with open("SampleID.txt", 'r') as file:
    sequenced_samples = file.readlines()
    sequenced_samples = [i.strip() for i in sequenced_samples]

In [7]:
len(sequenced_samples)

140

In [23]:
# these two samples appear in the sample list but not in the sequenced data

different_samples = set(sequenced_samples).symmetric_difference(set(df["SampleID"]))
different_samples

{'WSU_81', 'WSU_82'}

In [13]:
len(sequenced_samples)

140

In [24]:
df["Sequenced"] = df["SampleID"].isin(sequenced_samples)

# only take samples that we sequenced

seq_df = df[df["Sequenced"]]

In [25]:
seq_df

,Enrollment Number,Gender,Ethnicity,Healthy Control or HS?,SampleID,Sequenced
0,HS-42,F,Caucasian,HS,HS_42,True
1,HS-110,F,Caucasian,HS,HS_110,True
2,HS-129,F,African American,HS,HS_129,True
3,HS-132,F,Asian American,HS,HS_132,True
4,HS-157,F,Caucasian,HS,HS_157,True
...,...,...,...,...,...,...
137,WSU-119,M,African American,HS,WSU_119,True
138,WSU-122,F,African American,HS,WSU_122,True
139,WSU-124,F,African American,HS,WSU_124,True
140,WSU-126,F,African American,HS,WSU_126,True


In [26]:
# samples with no info

seq_df[seq_df["Healthy Control or HS?"] == '?']

,Enrollment Number,Gender,Ethnicity,Healthy Control or HS?,SampleID,Sequenced
88,HS-644,?,?,?,HS_644,True


In [27]:
seq_df["Healthy Control or HS?"].value_counts()

# remove samples which we don't know the health status of

seq_df_hshc = seq_df[seq_df["Healthy Control or HS?"] != '?'][["SampleID", "Gender", "Ethnicity", "Healthy Control or HS?"]]  
seq_df_hshc = seq_df_hshc.rename(columns={"Healthy Control or HS?" : "Condition"})

# total samples in list: 142
# total samples sequenced: 140 (unsequenced samples = WSU81, WSU82)
# Samples Missing Sample Info: 1 (HS_644)

In [28]:
df["Ethnicity"].value_counts()
df.loc[df["Ethnicity"].str.startswith("African American"),"Ethnicity"] = "African American"
df.loc[df["Ethnicity"].str.startswith("Hispanic"),"Ethnicity"] = "Hispanic"
df.loc[df["Ethnicity"].str.startswith("Asian"),"Ethnicity"] = "Asian"
df.loc[df["Ethnicity"].str.startswith("Caucasian"),"Ethnicity"] = "Caucasian"

In [29]:
df["Gender"].value_counts()
df.loc[df["Gender"].str.startswith("F"),"Gender"] = "F"
df.loc[df["Gender"].str.startswith("M"),"Gender"] = "M"

In [30]:
df.to_csv("pbmc_sample_list.csv", index = False)